In [1]:
import pyodbc
from import_function import import_scoring_data, import_data
from features_by_customer_type_3 import customer_type_features
from care_features_2 import care_features
# from features_by_customer_type import customer_type_features
conn = pyodbc.connect(dsn='VerticaProd')
ct='New'
response=['ABANDONED']
#  AND CUSTOMER_DEFINITION_ADJ='NEW TO TURBOTAX'
care_cont_features, care_bool_features, care_catag_features, care_cont_score_features, care_bool_score_features, care_catag_score_features=care_features(ct)
cont_features,bool_features,catag_features,cont_score_features,bool_score_features,catag_score_features=customer_type_features(ct)
data = "SELECT A.AUTH_ID,ABANDONED, AGE_TAXPAYER,RISK_FLAG_PY,ANC_PY,	AGI,	AMOUNT_INCOME_TAX_WITHHELD,	AMOUNT_INCOME_TAX,	AMOUNT_REFUND,	AMOUNT_SALARIES_AND_WAGES,	AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION,	AMOUNT_TAX,	AMOUNT_TOTAL_DEDUCTIONS,	AMOUNT_TOTAL_PAYMENTS,	AMOUNT_TOTAL_TAX,	COMPLETED_SKU,	COST_PER_CUST_LAG7,	COST_PER_CUST,	CUSTOMER_TYPE,	DMA_AREA,	FED_FORM_TYPE,	FILING_STATUS,	FIRST_COMPLETE_APP_TYPE,	FIRST_COMPLETE_DEVICE_TYPE,	FSCHA_FLAG,	IMPORT_TYPE,	NEAUTH_DEVICE_TYPE,	NUM_DEPENDENTS,	NUM_EXEMPTIONS,	NUM_W2,	PRS_SCORE,	REFUND_TRANSFER_FLAG,	REJECT_COUNT,	REQUIRED_TAKE_FLAG,	SESSIONS_TO_COMPLETE,	START_DEVICE_TYPE,	STATE_ATTACH_COUNT,	STUDENT_TAXPAYER,	TOTAL_REVENUE,	VAUTH_DEVICE_TYPE FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL A INNER JOIN CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SAMPLE B ON A.AUTH_ID=B.AUTH_ID WHERE TAX_YEAR IN (2015) "
data_PY = "SELECT A.AUTH_ID,ABANDONED, AGE_TAXPAYER,RISK_FLAG_PY,ANC_PY,	AGI,	AMOUNT_INCOME_TAX_WITHHELD,	AMOUNT_INCOME_TAX,	AMOUNT_REFUND,	AMOUNT_SALARIES_AND_WAGES,	AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION,	AMOUNT_TAX,	AMOUNT_TOTAL_DEDUCTIONS,	AMOUNT_TOTAL_PAYMENTS,	AMOUNT_TOTAL_TAX,	COMPLETED_SKU,	COST_PER_CUST_LAG7,	COST_PER_CUST,	CUSTOMER_TYPE,	DMA_AREA,	FED_FORM_TYPE,	FILING_STATUS,	FIRST_COMPLETE_APP_TYPE,	FIRST_COMPLETE_DEVICE_TYPE,	FSCHA_FLAG,	IMPORT_TYPE,	NEAUTH_DEVICE_TYPE,	NUM_DEPENDENTS,	NUM_EXEMPTIONS,	NUM_W2,	PRS_SCORE,	REFUND_TRANSFER_FLAG,	REJECT_COUNT,	REQUIRED_TAKE_FLAG,	SESSIONS_TO_COMPLETE,	START_DEVICE_TYPE,	STATE_ATTACH_COUNT,	STUDENT_TAXPAYER,	TOTAL_REVENUE,	VAUTH_DEVICE_TYPE FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL A INNER JOIN CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SAMPLE B ON A.AUTH_ID=B.AUTH_ID WHERE TAX_YEAR IN (2014) "
data_PY2 = "SELECT A.AUTH_ID,ABANDONED, AGE_TAXPAYER,RISK_FLAG_PY,ANC_PY,	AGI,	AMOUNT_INCOME_TAX_WITHHELD,	AMOUNT_INCOME_TAX,	AMOUNT_REFUND,	AMOUNT_SALARIES_AND_WAGES,	AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION,	AMOUNT_TAX,	AMOUNT_TOTAL_DEDUCTIONS,	AMOUNT_TOTAL_PAYMENTS,	AMOUNT_TOTAL_TAX,	COMPLETED_SKU,	COST_PER_CUST_LAG7,	COST_PER_CUST,	CUSTOMER_TYPE,	DMA_AREA,	FED_FORM_TYPE,	FILING_STATUS,	FIRST_COMPLETE_APP_TYPE,	FIRST_COMPLETE_DEVICE_TYPE,	FSCHA_FLAG,	IMPORT_TYPE,	NEAUTH_DEVICE_TYPE,	NUM_DEPENDENTS,	NUM_EXEMPTIONS,	NUM_W2,	PRS_SCORE,	REFUND_TRANSFER_FLAG,	REJECT_COUNT,	REQUIRED_TAKE_FLAG,	SESSIONS_TO_COMPLETE,	START_DEVICE_TYPE,	STATE_ATTACH_COUNT,	STUDENT_TAXPAYER,	TOTAL_REVENUE,	VAUTH_DEVICE_TYPE FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL A INNER JOIN CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SAMPLE B ON A.AUTH_ID=B.AUTH_ID WHERE TAX_YEAR IN (2013) "
care_data="SELECT DISTINCT  A.AUTH_ID,  TAX_YEAR, AXC_CARE,ENTERING_CONTACT_US_EXPERIENCE,TOTAL_AGENT_INTERACTION_SECONDS,CARE_REFERRER,	FIRST_HS_INTENT,	FIRST_SEARCH_MANUAL_LOCATION_DETAIL FROM CTG_ANALYTICS_WS.SM_CARE_DATA_MINING A INNER JOIN CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SAMPLE B ON A.AUTH_ID=B.AUTH_ID WHERE A.TAX_YEAR=2015 "
df_no_pca= import_data(data,care_data,data_PY,data_PY2, cont_features, bool_features,response, catag_features,care_cont_features, care_bool_features,care_catag_features)

print df_no_pca['ABANDONED'].unique()

c:\python27\lib\site-packages\pandas\core\frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


df_cont done
just_dummies done
df_trans done
df_cont_py done
just_dummies_py done
df_trans_py done
df_cont_py2 done
just_dummies_py2 done
df_trans_py2 done


import_function.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_cont_care.drop(care_bool_features, axis=1, inplace=True)


df_cont done
data_scaled done
       ABANDONED RISK_FLAG_PY ANC_PY FSCHA_FLAG REFUND_TRANSFER_FLAG  \
0           True        False   True      False                False   
1          False         True   True      False                False   
2          False         True   True      False                 True   
3           True         True   True      False                False   
4           True         True   True      False                 True   
5          False         True   True      False                 True   
6           True         True   True      False                False   
7          False         True   True      False                False   
8           True         True   True      False                False   
9           True         True   True      False                 True   
10         False        False   True      False                False   
11         False        False  False      False                False   
12          True        False  Fal

In [2]:
from bin_cont_features import bin_pca
j='False'
response='ABANDONED'
print df_no_pca[response].unique()
print df_no_pca.dtypes
dummy_pca, length_dict = bin_pca(df_no_pca,response, j)
scoring_list=list(dummy_pca)
print dummy_pca['ABANDONED'].unique()

[True False]
ABANDONED                                        bool
RISK_FLAG_PY                                     bool
ANC_PY                                           bool
FSCHA_FLAG                                       bool
REFUND_TRANSFER_FLAG                             bool
REQUIRED_TAKE_FLAG                               bool
STUDENT_TAXPAYER                                 bool
COMPLETED_SKU_200|Free TTO                       bool
COMPLETED_SKU_600|Paid Deluxe                    bool
COMPLETED_SKU_800|Paid Premier                   bool
COMPLETED_SKU_900|Paid Home and Business         bool
CUSTOMER_TYPE_Veteran Renewer                    bool
DMA_AREA_-1                                      bool
DMA_AREA_Abilene-Sweetwater                      bool
DMA_AREA_Albany  GA                              bool
DMA_AREA_Albany-Schenectady-Troy                 bool
DMA_AREA_Albuquerque-Santa Fe                    bool
DMA_AREA_Alexandria  LA                          bool
DMA_AREA_Alpena

In [3]:
response='ABANDONED'
ct='New'
from feature_clustering import feature_clustering
i='none'

df_no_pca,y, plsca= feature_clustering(dummy_pca,response, i)
print list(df_no_pca)
print df_no_pca['ABANDONED'].unique()




sample dataframe shape is:  (5000, 968)
{160: 21.958825122748276, 130: 22.361558493683415, 100: 22.265331499041167, 230: 20.68809817752091, 200: 21.719892904999917, 170: 22.426422885793375, 140: 21.429817451624679, 290: 21.680588103616131, 110: 22.15491433289673, 240: 21.343742788180837, 280: 21.250635431692405, 210: 21.752985819647161, 180: 20.71325848658763, 270: 21.94882083572401, 150: 23.276430997440084, 120: 22.126845935022349, 260: 22.764148510344686, 250: 22.673045865883655, 220: 21.975021271215024, 190: 21.93066928834434}
the best # of components is: 150
['ABANDONED', 'RISK_FLAG_PY', 'ANC_PY', 'FSCHA_FLAG', 'REFUND_TRANSFER_FLAG', 'REQUIRED_TAKE_FLAG', 'STUDENT_TAXPAYER', u'COMPLETED_SKU_200|Free TTO', u'COMPLETED_SKU_600|Paid Deluxe', u'COMPLETED_SKU_800|Paid Premier', u'COMPLETED_SKU_900|Paid Home and Business', u'CUSTOMER_TYPE_Veteran Renewer', 'DMA_AREA_-1', u'DMA_AREA_Abilene-Sweetwater', u'DMA_AREA_Albany  GA', u'DMA_AREA_Albany-Schenectady-Troy', u'DMA_AREA_Albuquerque-S

In [4]:
from clustering import obs_clustering
clust='False'
df=df_no_pca
print df
df, df_w_labels, df_w_clust=obs_clustering(df,response,clust)
print df
print df['ABANDONED'].unique()

       ABANDONED RISK_FLAG_PY ANC_PY FSCHA_FLAG REFUND_TRANSFER_FLAG  \
0           True        False   True      False                False   
1          False         True   True      False                False   
2          False         True   True      False                 True   
3           True         True   True      False                False   
4           True         True   True      False                 True   
5          False         True   True      False                 True   
6           True         True   True      False                False   
7          False         True   True      False                False   
8           True         True   True      False                False   
9           True         True   True      False                 True   
10         False        False   True      False                False   
11         False        False  False      False                False   
12          True        False  False      False                F

In [5]:
from get_arrays import get_arrays
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
i='False'
j='False'
dummy_pca=df_no_pca
response = 'ABANDONED'
x, y = get_arrays(dummy_pca, response, i, j)
print x.shape
x2=x.copy()
from sklearn.feature_selection import SelectPercentile, f_classif
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sel = VarianceThreshold(threshold=(.7 * (1 - .7)))
reduced_x=sel.fit_transform(x2)
inverse_transform=sel.inverse_transform(reduced_x)

reduced_df=pd.DataFrame(inverse_transform, columns=list(x2))

for column in reduced_df:
    if len(reduced_df[column].unique())==1:
        reduced_df.drop([column], axis=1, inplace=True)
print reduced_df.shape
selector = SelectPercentile(f_classif, percentile=20)

selector.fit(reduced_df, y)
print selector.pvalues_

# df_pvalues=pd.DataFrame(selector.pval,columns=['p_value'], index=list(x))
df_pvalues=pd.DataFrame(selector.pvalues_,columns=['p_value'], index=list(reduced_df))
print df_pvalues
sig_features=df_pvalues[df_pvalues['p_value'] <=0.05]
sig_features.reset_index(['index'], inplace=True)
sig_features.drop(['p_value'], axis=1, inplace=True)
print sig_features


reduced_df=reduced_df[sig_features['index'].tolist()]
# print x.shape
x=reduced_df
reduced_df.to_csv(path_or_buf='x.txt', index=True)

data = pd.read_csv('x.txt', sep=",")


print x.shape

(500000, 967)
(500000, 40)
[  2.27727287e-219   5.71731961e-079   8.52293869e-293   3.78220696e-018
   0.00000000e+000   3.14254181e-047   3.92913431e-012   5.37586797e-019
   3.27500000e-224   1.21934811e-296   1.08290364e-016   1.92512210e-301
   0.00000000e+000   1.22744933e-001   0.00000000e+000   3.80677888e-023
   4.13821551e-167   1.01817146e-066   0.00000000e+000   0.00000000e+000
   1.14139910e-017   1.07127818e-033   1.22856127e-011   0.00000000e+000
   5.74550479e-003   0.00000000e+000   0.00000000e+000   0.00000000e+000
   2.68207934e-007   1.32503348e-181   0.00000000e+000   0.00000000e+000
   1.50048287e-002   1.49899445e-251   0.00000000e+000   2.54680959e-319
   3.89618342e-027   0.00000000e+000   2.07213924e-067   1.77643869e-015]
                                              p_value
COMPLETED_SKU_200|Free TTO              2.277273e-219
COMPLETED_SKU_600|Paid Deluxe            5.717320e-79
FED_FORM_TYPE_F1040                     8.522939e-293
FED_FORM_TYPE_F1040A      

c:\python27\lib\site-packages\ipykernel\__main__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


                                     index
0               COMPLETED_SKU_200|Free TTO
1            COMPLETED_SKU_600|Paid Deluxe
2                      FED_FORM_TYPE_F1040
3                     FED_FORM_TYPE_F1040A
4       FILING_STATUS_MarriedFilingJointly
5                     FILING_STATUS_Single
6                       IMPORT_TYPE_Import
7                      IMPORT_TYPE_Partial
8                           PRS_SCORE_10.0
9                             PRS_SCORE_-1
10                           FSCHA_FLAG_py
11                 REFUND_TRANSFER_FLAG_py
12           COMPLETED_SKU_200|Free TTO_py
13          CUSTOMER_TYPE_1st Year Renewer
14                  FED_FORM_TYPE_F1040_py
15                 FED_FORM_TYPE_F1040A_py
16   FILING_STATUS_MarriedFilingJointly_py
17                 FILING_STATUS_Single_py
18                       IMPORT_TYPE_-1_py
19                   IMPORT_TYPE_Import_py
20                  IMPORT_TYPE_Partial_py
21                       PRS_SCORE_10.0_py
22         

In [6]:
print y.head(), x.head()
y.columns=[response]
print y.head(), x.head()
from time import time
start=time()
from model_algorithm import algorithm
response = 'ABANDONED'
models, name = algorithm(x, y, response)
end=time()
print (end-start)/60

0     True
1    False
2    False
3     True
4     True
Name: ABANDONED, dtype: bool    COMPLETED_SKU_200|Free TTO  COMPLETED_SKU_600|Paid Deluxe  \
0                         1.0                            0.0   
1                         1.0                            0.0   
2                         1.0                            0.0   
3                         1.0                            0.0   
4                         1.0                            0.0   

   FED_FORM_TYPE_F1040  FED_FORM_TYPE_F1040A  \
0                  0.0                   0.0   
1                  0.0                   0.0   
2                  0.0                   1.0   
3                  0.0                   0.0   
4                  0.0                   0.0   

   FILING_STATUS_MarriedFilingJointly  FILING_STATUS_Single  \
0                                 0.0                   0.0   
1                                 0.0                   1.0   
2                                 0.0                

c:\python27\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Decision_Tree  precision: 0.217540450216  recall: 0.587071397354  f1: 0.31744940489  auc: 0.580659890584  tn: 71684  fp: 53147  fn: 10393  tp: 14776
Random_Forest  precision: 0.217741378119  recall: 0.24776847155  f1: 0.231786495981  auc: 0.533022946048  tn: 101932  fp: 22637  fn: 19130  tp: 6301
NeuralNetwork  precision: 0.549450549451  recall: 0.0390563974379  f1: 0.0729288214702  auc: 0.516232272805  tn: 30894  fp: 205  fn: 6151  tp: 250
logistic_regression  precision: 0.224573420586  recall: 0.580668116229  f1: 0.323884497073  auc: 0.58699016653  tn: 73989  fp: 50716  fn: 10607  tp: 14688
45.5446166674


In [9]:
import pandas as pd
import numpy as np
import pickle
from time import time
ct='New'
response=['ABANDONED']
filename = 'finalized_model.sav'
filename2 ='name.sav'
filename3 = 'bin_model.sav'
filename4 = 'feature_selection.sav'
filename5 = 'get_arrays.sav'
# filename5 ='score_model.sav'
# from care_features_2 import care_features
from predicting_code import transform_to_pca, bin_pca_score_set, get_scoring_arrays, score_set_feature_selection, predict,get_scoring_arrays
from import_function import import_scoring_data
from features_by_customer_type_3 import customer_type_features
defection=pd.DataFrame(columns=['AUTH_ID','Decision_Tree_defect_prob','Decision_Tree_flag','Decision_Tree_retain_prob','logistic_regression_defect_prob','logistic_regression_flag','logistic_regression_retain_prob'
])
cont_features,bool_features,catag_features,cont_score_features,bool_score_features,catag_score_features=customer_type_features(ct)

# load the model from disk
models = pickle.load(open(filename, 'rb'))
name = pickle.load(open(filename2, 'rb'))
length_dict = pickle.load(open(filename3, 'rb'))
plsca = pickle.load(open(filename4, 'rb'))

x = pd.read_csv('x.txt', sep=",")
# x.drop(['AUTH_ID'], axis=1, inplace=True)
# scoring_data = "SELECT  A.AUTH_ID, AGE_TAXPAYER,RISK_FLAG_PY,ANC_PY,	AGI,	AMOUNT_INCOME_TAX_WITHHELD,	AMOUNT_INCOME_TAX,	AMOUNT_REFUND,	AMOUNT_SALARIES_AND_WAGES,	AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION,	AMOUNT_TAX,	AMOUNT_TOTAL_DEDUCTIONS,	AMOUNT_TOTAL_PAYMENTS,	AMOUNT_TOTAL_TAX,	COMPLETED_SKU,	COST_PER_CUST_LAG7,	COST_PER_CUST,	CUSTOMER_TYPE,	DMA_AREA,	FED_FORM_TYPE,	FILING_STATUS,	FIRST_COMPLETE_APP_TYPE,	FIRST_COMPLETE_DEVICE_TYPE,	FSCHA_FLAG,	IMPORT_TYPE,	NEAUTH_DEVICE_TYPE,	NUM_DEPENDENTS,	NUM_EXEMPTIONS,	NUM_W2,	PRS_SCORE,	REFUND_TRANSFER_FLAG,	REJECT_COUNT,	REQUIRED_TAKE_FLAG,	SESSIONS_TO_COMPLETE,	START_DEVICE_TYPE,	STATE_ATTACH_COUNT,	STUDENT_TAXPAYER,	TOTAL_REVENUE,	VAUTH_DEVICE_TYPE FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL A INNER JOIN CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SCORE_SAMPLE B ON A.AUTH_ID=B.AUTH_ID WHERE TAX_YEAR=2016"
# scoring_data_PY = "SELECT  A.AUTH_ID, AGE_TAXPAYER,RISK_FLAG_PY,ANC_PY,	AGI,	AMOUNT_INCOME_TAX_WITHHELD,	AMOUNT_INCOME_TAX,	AMOUNT_REFUND,	AMOUNT_SALARIES_AND_WAGES,	AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION,	AMOUNT_TAX,	AMOUNT_TOTAL_DEDUCTIONS,	AMOUNT_TOTAL_PAYMENTS,	AMOUNT_TOTAL_TAX,	COMPLETED_SKU,	COST_PER_CUST_LAG7,	COST_PER_CUST,	CUSTOMER_TYPE,	DMA_AREA,	FED_FORM_TYPE,	FILING_STATUS,	FIRST_COMPLETE_APP_TYPE,	FIRST_COMPLETE_DEVICE_TYPE,	FSCHA_FLAG,	IMPORT_TYPE,	NEAUTH_DEVICE_TYPE,	NUM_DEPENDENTS,	NUM_EXEMPTIONS,	NUM_W2,	PRS_SCORE,	REFUND_TRANSFER_FLAG,	REJECT_COUNT,	REQUIRED_TAKE_FLAG,	SESSIONS_TO_COMPLETE,	START_DEVICE_TYPE,	STATE_ATTACH_COUNT,	STUDENT_TAXPAYER,	TOTAL_REVENUE,	VAUTH_DEVICE_TYPE FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL A INNER JOIN CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SCORE_SAMPLE B ON A.AUTH_ID=B.AUTH_ID WHERE TAX_YEAR=2015 "
# scoring_data_PY2 = "SELECT A.AUTH_ID, AGE_TAXPAYER,RISK_FLAG_PY,ANC_PY,	AGI,	AMOUNT_INCOME_TAX_WITHHELD,	AMOUNT_INCOME_TAX,	AMOUNT_REFUND,	AMOUNT_SALARIES_AND_WAGES,	AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION,	AMOUNT_TAX,	AMOUNT_TOTAL_DEDUCTIONS,	AMOUNT_TOTAL_PAYMENTS,	AMOUNT_TOTAL_TAX,	COMPLETED_SKU,	COST_PER_CUST_LAG7,	COST_PER_CUST,	CUSTOMER_TYPE,	DMA_AREA,	FED_FORM_TYPE,	FILING_STATUS,	FIRST_COMPLETE_APP_TYPE,	FIRST_COMPLETE_DEVICE_TYPE,	FSCHA_FLAG,	IMPORT_TYPE,	NEAUTH_DEVICE_TYPE,	NUM_DEPENDENTS,	NUM_EXEMPTIONS,	NUM_W2,	PRS_SCORE,	REFUND_TRANSFER_FLAG,	REJECT_COUNT,	REQUIRED_TAKE_FLAG,	SESSIONS_TO_COMPLETE,	START_DEVICE_TYPE,	STATE_ATTACH_COUNT,	STUDENT_TAXPAYER,	TOTAL_REVENUE,	VAUTH_DEVICE_TYPE FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL A INNER JOIN CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SCORE_SAMPLE B ON A.AUTH_ID=B.AUTH_ID WHERE TAX_YEAR=2014 "
# care_score_data="SELECT DISTINCT  A.AUTH_ID, TAX_YEAR, AXC_CARE,ENTERING_CONTACT_US_EXPERIENCE,TOTAL_AGENT_INTERACTION_SECONDS,CARE_REFERRER,	FIRST_HS_INTENT,	FIRST_SEARCH_MANUAL_LOCATION_DETAIL FROM CTG_ANALYTICS_WS.SM_CARE_DATA_MINING A INNER JOIN CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SCORE_SAMPLE B ON A.AUTH_ID=B.AUTH_ID WHERE A.TAX_YEAR=2016 "


for m in range(1, 15266274, 1000000):
# for m in range(1, 10000, 11117688):
    n=m+1000000
    s=m
    print m
    a=[]
    b=[]
    common_ftrs=[]
    missing_a=[]
    
    
    start = time() 
    scoring_data = "SELECT * FROM CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SCORE_SAMPLE WHERE TAX_YEAR=2016 AND rank between %d and %d order by rank"%(m,n)
    scoring_data_PY = "SELECT * FROM CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SCORE_SAMPLE WHERE TAX_YEAR=2015 AND rank between %d and %d order by rank"%(m,n)
    scoring_data_PY2 = "SELECT * FROM CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SCORE_SAMPLE WHERE TAX_YEAR=2014 AND rank between %d and %d order by rank"%(m,n)
    care_score_data="SELECT DISTINCT  A.AUTH_ID, TAX_YEAR, AXC_CARE,ENTERING_CONTACT_US_EXPERIENCE,TOTAL_AGENT_INTERACTION_SECONDS,CARE_REFERRER,	FIRST_HS_INTENT,	FIRST_SEARCH_MANUAL_LOCATION_DETAIL FROM CTG_ANALYTICS_WS.SM_CARE_DATA_MINING A INNER JOIN CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SCORE_SAMPLE B ON A.AUTH_ID=B.AUTH_ID WHERE A.TAX_YEAR=2016 "
    scoring_df=import_scoring_data(scoring_data,care_score_data,scoring_data_PY, scoring_data_PY2, cont_score_features, bool_score_features, catag_score_features, care_cont_score_features, care_bool_score_features, care_catag_score_features)

    scoring_df2=import_scoring_data(data_s, cont_score_features, bool_score_features, catag_score_features)
    scoring_df=scoring_df2.copy()
    i, j, k=['False','False','none' ]   
    scoring_df.reset_index(['AUTH_ID'], inplace=True)
    index=scoring_df['AUTH_ID']
    scoring_df.drop(['AUTH_ID'], axis=1, inplace=True)    
    scoring_df_trans = bin_pca_score_set(scoring_df, length_dict, j)
    print scoring_df_trans.shape, x.shape
    x2=x.copy()
    a=list(x2)
    b=list(scoring_df_trans)
    final=list(set(b)- set(common_ftrs))
#     print " Diff between score data and common ftrs:", final
    common_ftrs=list(set(a) & set(b))  
    missing_a=list(set(a)- set(common_ftrs))
    print "common_ftrs:", len(common_ftrs)," missing_a:", len(missing_a) , " missing_features:" , missing_a   
    m=scoring_df_trans.shape[0]
    zeros_df=pd.DataFrame(np.zeros((m, len(missing_a))), columns=missing_a).astype('bool')
    final_scoring_df=pd.concat([zeros_df,scoring_df_trans[common_ftrs],index ], axis=1)
    final_scoring_df.set_index('AUTH_ID', inplace=True)
    print final_scoring_df.shape, list(final_scoring_df)
    final_scoring_df.drop(['Unnamed: 0'], axis=1, inplace=True)
    scoring_df_trans=final_scoring_df
    x_score, index=score_set_feature_selection(scoring_df_trans,plsca, k)
    
    x_score.set_index('AUTH_ID', inplace=True) 
    print x_score.head()
    df_p=predict(models,name,x_score, index)
    df_run=df_p.copy()
    df_run.append(defection, ignore_index=True)
    print df_p.head()
    df_p.to_csv(path_or_buf='defection_model_LOYAL_%d.txt' % s, index=True)
    end = time()
    print (end-start)/60
defection.to_csv(path_or_buf='defection_model_LOYAL.txt', index=True)

IOError: [Errno 2] No such file or directory: 'bin_model.sav'

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.neural_network import MLPClassifier
import numpy as np
print y.unique()
train_sizes, train_scores, valid_scores = learning_curve(MLPClassifier(alpha=1e-5, random_state=np.random.RandomState(0)), x, y, train_sizes=[ 0.01, 0.05, 0.1, 0.5,0.75, 1. ], cv=5, scoring='roc_auc')

for i in train_scores:
    print np.mean(i)
for K in valid_scores:
    print np.mean(K)


In [ ]:
import pandas as pd
from data_intersection import scoring_data_intersection
y_df=df_no_pca[response]
print scoring_df.head()

data, _ = scoring_data_intersection(df_no_pca, scoring_df)
print data.shape
print scoring_df.head()
df_pca=pd.concat([y_df, data], axis=1)
# df_pca.reset_index(['CUSTOMER_KEY'], inplace=True)
print df_pca.shape
